In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# https://huggingface.co/EleutherAI/polyglot-ko-5.8b
# https://colab.research.google.com/github/jagaldol/chat-foodie/blob/main/chatbot/fine-tuning/KoAlpaca_QLoRA_ChatFoodie_Model_FineTuning.ipynb#scrollTo=E0Nl5mWL0k2T

!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.1 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [ ]:
model_id = "beomi/KoAlpaca-Polyglot-5.8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer1 = AutoTokenizer.from_pretrained(model_id)
model1 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/소제목/자기소개서소제목데이터셋.xlsx')
data['title'] = data['title'].str.replace('\xa0', ' ')
data['title'] = data['title'].str.replace("\'", '')
data['content'] = data['content'].str.replace('\xa0', ' ')
data['content'] = data['content'].str.replace("\'", '')
data['content'] = data['content'].str.replace("\n", ' ')

data.head()

,content,title
0,저는 고객의 니즈와 브랜드 강점을 효과적으로 연결하는 역량을 통해 야마하뮤직코리아에...,지피지기면 백전백승
1,저는 야마하뮤직코리아에서 게임 커뮤니티 회원들을 타깃으로 야마하 사운드바 팝업스토어...,니즈를 파악하기 위해 고객에게 더 가까이
2,저는 대학생때 발레를 배우기 시작했습니다. 처음에는 따라하는 것조차 벅차 그만두고 ...,초보에서 아마추어까지
3,저는 상경계적 직무 역량에 강점을 가지고 있습니다. 상경계 직무 지식을 쌓기 위해 ...,"상경계 역량 강점, 법학 역량 개발"
4,대학 시절 전공과목을 수강하면서 사회적 기업에 대해 조사하고 보고서를 작성하는 팀 ...,역지사지의 관점으로 상대방 이해하기


In [ ]:
text = '''
행정업무는 매뉴얼 이해와 정확한 민원 응대가 중요합니다.
또한 다른 부서와의 유기적인 연결을 고려하여 자발적이고 융통성 있게 일을 처리하는 능력이 필요합니다.
저는 어려운 도전에 긍정적으로 대처하고 업무를 책임감 있게 수행할 수 있습니다.
한국환경공단의 인턴으로 근무하며 전공지식의 부족함을 보완하기 위해 노력했습니다.
업무 가이드북과 현장 교육을 통해 전문지식을 습득하고 출장도 적극적으로 동행한 결과 다른 인턴보다 빠른 업무에 참여할 수 있었습니다.
저는 정확한 문서 처리능력을 가지고 있습니다. 삼성멀티캠퍼스에서 ProDs 데이터 교육을 통해 데이터 전처리 분석을 한 경험이 있습니다.
이를 통해 대량의 데이터를 원활히 다룰 수 있습니다.
또 공공기관에서 중요한 개인정보 관리에 필요한 능력을 갖추고 있습니다.
정보보안과 데이터보안에 대해 학습했고, 데이터보안 공모전 참여를 통해 학습을 넘어 정보보호와 데이터관리 실무 경험을 쌓았습니다.
정보보호 강화와 고객 복지서비스 향상을 위한 정확한 통계조사 및 데이터 분석 관리와 정보보호 관리에 적합한 인재가 되겠습니다.
'''

In [ ]:
system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적인 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {text}\n\n### 소제목:\n"
inputs = tokenizer1(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model1.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer1.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적인 소제목을 생성해줘.

### 자기소개서: 
행정업무는 매뉴얼 이해와 정확한 민원 응대가 중요합니다.
또한 다른 부서와의 유기적인 연결을 고려하여 자발적이고 융통성 있게 일을 처리하는 능력이 필요합니다.
저는 어려운 도전에 긍정적으로 대처하고 업무를 책임감 있게 수행할 수 있습니다.
한국환경공단의 인턴으로 근무하며 전공지식의 부족함을 보완하기 위해 노력했습니다.
업무 가이드북과 현장 교육을 통해 전문지식을 습득하고 출장도 적극적으로 동행한 결과 다른 인턴보다 빠른 업무에 참여할 수 있었습니다.
저는 정확한 문서 처리능력을 가지고 있습니다. 삼성멀티캠퍼스에서 ProDs 데이터 교육을 통해 데이터 전처리 분석을 한 경험이 있습니다.
이를 통해 대량의 데이터를 원활히 다룰 수 있습니다.
또 공공기관에서 중요한 개인정보 관리에 필요한 능력을 갖추고 있습니다.
정보보안과 데이터보안에 대해 학습했고, 데이터보안 공모전 참여를 통해 학습을 넘어 정보보호와 데이터관리 실무 경험을 쌓았습니다.
정보보호 강화와 고객 복지서비스 향상을 위한 정확한 통계조사 및 데이터 분석 관리와 정보보호 관리에 적합한 인재가 되겠습니다.


### 소제목:
자기소개서 작성 방법과 소제목 작성 방법을 알려주세요.

### 맥락:
안녕하세요. 귀하의 자기소개서에 대한 답변입니다.
[자기소개서 작성법]
1. 성장과정유년


In [ ]:
dataset = Dataset.from_pandas(data)
eos_token = "[EOS]"
tokenizer1.add_special_tokens({"eos_token": eos_token})

def format_data(example):
    formatted_text = f"### 자기소개서: {example['content']}\n\n### 소제목: {example['title']} ['EOS']"
    return {"formatted": formatted_text}

dataset = dataset.map(format_data)

dts = DatasetDict({
    'train': dataset
})

print(dts)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['content', 'title', 'formatted'],
        num_rows: 200
    })
})


In [ ]:
dts = dts.map(lambda samples: tokenizer1(samples["formatted"]), batched=True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
dts

DatasetDict({
    train: Dataset({
        features: ['content', 'title', 'formatted', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 200
    })
})

In [ ]:
dts['train'][0]['formatted']

'### 자기소개서: 저는 고객의 니즈와 브랜드 강점을 효과적으로 연결하는 역량을 통해 야마하뮤직코리아에서 신세대 유입 프로젝트를 성공적으로 이끈 경험이 있습니다. 당시 야마하 오디오는 주로 기성세대 매니아층을 타깃으로 하고 있었고, 그로 인해 다양한 세대를 타깃으로 하는 타 브랜드에 비해 인지도가 낮았습니다. 이를 해결하기 위해 대학연합문화마케팅학회의 경험을 바탕으로 신세대 유입 프로젝트를 기획했습니다. 대학연합문화마케팅학회에서 다양한 기업을 분석하며 IMC 전략을 수립했는데, 이를 통해 "지피지기면 백전백승"이라는 교훈을 얻었습니다. 즉, 타깃 고객의 니즈를 정확히 파악하고 이를 기업의 강점과 효과적으로 연결하는 것이 성공적인 마케팅의 열쇠라는 것을 깨달았습니다. 이를 바탕으로 야마하뮤직코리아에서 신세대 중 20대를 타깃으로 두고 마케팅 방안을 기획하였습니다. 먼저 20대가 많이 찾는 취미 공유 플랫폼에 직접 참여하여 요즘 대학생들이 어떠한 것에 관심을 많이 가지는지 조사하였고 이를 통해 자신의 생각을 나눌 수 있는 자리를 선호한다는 점을 알게 되었습니다. 이러한 점과 하이엔드 스피커로 최고 음질로 음악을 전달하는 브랜드의 강점을 연결하여 좋아하는 음악과 자신의 생각을 나눌 수 있는 자리인 건국대학교 청음회를 기획하였습니다.  주제는 나의 대학생활, 최애 영화를 소개합니다, 미리 크리스마스로 대학생들이 많은 관심을 가질만한 주제로 선정했습니다. 또한 사전에 신청한 음악과 사연을 들으며 실시간으로 서로의 생각을 자유롭게 공유할 수 있는 모바일 게시판을 열었습니다. 활발한 대화를 돕기 위해 청음회 몰입감을 높이고자 하였고 각 주제에 맞추어 청음회 내부를 꾸미기도 했습니다. 이 청음회는 대학생들로부터 큰 관심을 받았으며, 정원이 약 30명인 청음회에서 매회차마다 20개이상의 리뷰 콘텐츠가 제작되었습니다. 타깃고객의 니즈와 브랜드의 강점을 정확히 파악하여 연결함으로써 얻은 성과였습니다.\n\n### 소제목: 지피지기면 백전백승 [\'EOS\']'

In [ ]:
model1.gradient_checkpointing_enable()
model1 = prepare_model_for_kbit_training(model1)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model1 = get_peft_model(model1, config)
print_trainable_parameters(model1)

trainable params: 3670016 || all params: 3070156800 || trainable%: 0.11953838970048696


In [ ]:
### train
tokenizer1.pad_token = tokenizer1.eos_token

trainer = transformers.Trainer(
    model=model1,
    train_dataset=dts["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        max_steps=300,  # num_train_epochs는 명시하지 않음
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        logging_dir="./logs",
        output_dir="outputs",
        run_name="my_training_run",  # run_name을 명시적으로 다르게 설정
        report_to="none"  # wandb 로깅 비활성화
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer1, mlm=False),
)
model1.config.use_cache = False
trainer.train()

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,4.209500
20,3.919900
30,3.322600
40,2.949600
50,2.710500
60,2.650100
70,2.624900
80,2.559800
90,2.583700
100,2.520700


TrainOutput(global_step=300, training_loss=2.501996955871582, metrics={'train_runtime': 2758.3395, 'train_samples_per_second': 0.87, 'train_steps_per_second': 0.109, 'total_flos': 3.992019348396442e+16, 'train_loss': 2.501996955871582, 'epoch': 12.0})

In [ ]:
model1.save_pretrained('/content/drive/MyDrive/소제목/model')
tokenizer1.save_pretrained('/content/drive/MyDrive/소제목/model')

('/content/drive/MyDrive/소제목/model/tokenizer_config.json',
 '/content/drive/MyDrive/소제목/model/special_tokens_map.json',
 '/content/drive/MyDrive/소제목/model/tokenizer.json')

In [ ]:
model1.eval()
model1.config.use_cache = True

In [ ]:
system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {text}\n\n### 소제목:\n"
inputs = tokenizer1(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model1.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer1.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
행정업무는 매뉴얼 이해와 정확한 민원 응대가 중요합니다.
또한 다른 부서와의 유기적인 연결을 고려하여 자발적이고 융통성 있게 일을 처리하는 능력이 필요합니다.
저는 어려운 도전에 긍정적으로 대처하고 업무를 책임감 있게 수행할 수 있습니다.
한국환경공단의 인턴으로 근무하며 전공지식의 부족함을 보완하기 위해 노력했습니다.
업무 가이드북과 현장 교육을 통해 전문지식을 습득하고 출장도 적극적으로 동행한 결과 다른 인턴보다 빠른 업무에 참여할 수 있었습니다.
저는 정확한 문서 처리능력을 가지고 있습니다. 삼성멀티캠퍼스에서 ProDs 데이터 교육을 통해 데이터 전처리 분석을 한 경험이 있습니다.
이를 통해 대량의 데이터를 원활히 다룰 수 있습니다.
또 공공기관에서 중요한 개인정보 관리에 필요한 능력을 갖추고 있습니다.
정보보안과 데이터보안에 대해 학습했고, 데이터보안 공모전 참여를 통해 학습을 넘어 정보보호와 데이터관리 실무 경험을 쌓았습니다.
정보보호 강화와 고객 복지서비스 향상을 위한 정확한 통계조사 및 데이터 분석 관리와 정보보호 관리에 적합한 인재가 되겠습니다.


### 소제목:
행정업무에 능통하게: 공공기관에서 사무업무를 맡아 능통하게: 데이터 및 정보보호에 능통하게

### 소제목: 공공기관에서 사무업무를 능통하게:


In [ ]:
# 저장한 모델 불러오고 적용해보기

In [ ]:
model_id = "/content/drive/MyDrive/소제목/model/epoch300"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/36.8k [00:00<?, ?B/s]

model-00001-of-00013.safetensors:   0%|          | 0.00/926M [00:00<?, ?B/s]

model-00002-of-00013.safetensors:   0%|          | 0.00/952M [00:00<?, ?B/s]

...

model-00011-of-00013.safetensors:   0%|          | 0.00/952M [00:00<?, ?B/s]

model-00012-of-00013.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

model-00013-of-00013.safetensors:   0%|          | 0.00/515M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적인 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {text}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적인 소제목을 생성해줘.

### 자기소개서: 
행정업무는 매뉴얼 이해와 정확한 민원 응대가 중요합니다.
또한 다른 부서와의 유기적인 연결을 고려하여 자발적이고 융통성 있게 일을 처리하는 능력이 필요합니다.
저는 어려운 도전에 긍정적으로 대처하고 업무를 책임감 있게 수행할 수 있습니다.
한국환경공단의 인턴으로 근무하며 전공지식의 부족함을 보완하기 위해 노력했습니다.
업무 가이드북과 현장 교육을 통해 전문지식을 습득하고 출장도 적극적으로 동행한 결과 다른 인턴보다 빠른 업무에 참여할 수 있었습니다.
저는 정확한 문서 처리능력을 가지고 있습니다. 삼성멀티캠퍼스에서 ProDs 데이터 교육을 통해 데이터 전처리 분석을 한 경험이 있습니다.
이를 통해 대량의 데이터를 원활히 다룰 수 있습니다.
또 공공기관에서 중요한 개인정보 관리에 필요한 능력을 갖추고 있습니다.
정보보안과 데이터보안에 대해 학습했고, 데이터보안 공모전 참여를 통해 학습을 넘어 정보보호와 데이터관리 실무 경험을 쌓았습니다.
정보보호 강화와 고객 복지서비스 향상을 위한 정확한 통계조사 및 데이터 분석 관리와 정보보호 관리에 적합한 인재가 되겠습니다.


### 소제목:
행정 업무에 능통하며, 공공기관에서 업무를 수행하기 위한 능력을 보유한 인재 ['EOS']

### 소제목: 
행정 업무에 능통하며, 공공기관에서


In [ ]:
text1 = '''
DRAM 메모리에서 금속산화물의 활용에 주목하여, 인듐 갈륨 아연 산화물(IGZO) 절연체를 개발하는 연구를 수행했습니다.
다양한 첨가물 조건을 조절하며 공정을 통해 IGZO 절연체를 합성하고, XPS 및 SEM을 통해 특성을 분석했습니다.
초기 단계에서 단일 공정으로 진행했으나, SEM 분석 결과 균일성에 문제가 발견되어, 이를 해결하기 위해 두 번의 공정을 적용하여 표면 품질을 개선했습니다.
또한, 필름의 전기적 성능을 향상시키기 위해 특정 결함 구조의 영향을 분석했습니다.
이를 위해 XPS를 활용하여 최적의 열처리 조건을 도출하였습니다.
이 과정을 통해 저온 공정에서 낮은 누설 전류와 넓은 주파수 범위에서 안정적인 유전상수를 가진 절연체를 성공적으로 제작할 수 있었습니다.
이러한 성과는 IGZO 절연체의 다양한 열처리 방법을 탐색하고, DRAM의 절연층으로 활용하는 후속 연구로 이어졌습니다.
소자 분석 결과를 바탕으로 공정을 최적화한 경험은 공정 변수 조정 및 재료 특성 분석의 중요성을 깊이 이해하게 해주었고,
이를 바탕으로 양산 환경에서 효율적인 생산성과 고품질 제품 공급에 기여하고 싶습니다.
'''

system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {text1}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
DRAM 메모리에서 금속산화물의 활용에 주목하여, 인듐 갈륨 아연 산화물(IGZO) 절연체를 개발하는 연구를 수행했습니다. 
다양한 첨가물 조건을 조절하며 공정을 통해 IGZO 절연체를 합성하고, XPS 및 SEM을 통해 특성을 분석했습니다. 
초기 단계에서 단일 공정으로 진행했으나, SEM 분석 결과 균일성에 문제가 발견되어, 이를 해결하기 위해 두 번의 공정을 적용하여 표면 품질을 개선했습니다.
또한, 필름의 전기적 성능을 향상시키기 위해 특정 결함 구조의 영향을 분석했습니다. 
이를 위해 XPS를 활용하여 최적의 열처리 조건을 도출하였습니다. 
이 과정을 통해 저온 공정에서 낮은 누설 전류와 넓은 주파수 범위에서 안정적인 유전상수를 가진 절연체를 성공적으로 제작할 수 있었습니다. 
이러한 성과는 IGZO 절연체의 다양한 열처리 방법을 탐색하고, DRAM의 절연층으로 활용하는 후속 연구로 이어졌습니다.
소자 분석 결과를 바탕으로 공정을 최적화한 경험은 공정 변수 조정 및 재료 특성 분석의 중요성을 깊이 이해하게 해주었고, 
이를 바탕으로 양산 환경에서 효율적인 생산성과 고품질 제품 공급에 기여하고 싶습니다.


### 소제목:
 공정 조건 변경 및 불량 샘플 발생으로 공정 최적화 ['EOS']

### 소제목: 
 DRAM 메모리에서 절연체의 품질 관리 ['EOS']

### 소


In [ ]:
# https://linkareer.com/cover-letter/33942?keyword=DRAM%20%EB%A9%94%EB%AA%A8%EB%A6%AC&page=1&sort=PASSED_AT&tab=all
system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {text1}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
DRAM 메모리에서 금속산화물의 활용에 주목하여, 인듐 갈륨 아연 산화물(IGZO) 절연체를 개발하는 연구를 수행했습니다. 
다양한 첨가물 조건을 조절하며 공정을 통해 IGZO 절연체를 합성하고, XPS 및 SEM을 통해 특성을 분석했습니다. 
초기 단계에서 단일 공정으로 진행했으나, SEM 분석 결과 균일성에 문제가 발견되어, 이를 해결하기 위해 두 번의 공정을 적용하여 표면 품질을 개선했습니다.
또한, 필름의 전기적 성능을 향상시키기 위해 특정 결함 구조의 영향을 분석했습니다. 
이를 위해 XPS를 활용하여 최적의 열처리 조건을 도출하였습니다. 
이 과정을 통해 저온 공정에서 낮은 누설 전류와 넓은 주파수 범위에서 안정적인 유전상수를 가진 절연체를 성공적으로 제작할 수 있었습니다. 
이러한 성과는 IGZO 절연체의 다양한 열처리 방법을 탐색하고, DRAM의 절연층으로 활용하는 후속 연구로 이어졌습니다.
소자 분석 결과를 바탕으로 공정을 최적화한 경험은 공정 변수 조정 및 재료 특성 분석의 중요성을 깊이 이해하게 해주었고, 
이를 바탕으로 양산 환경에서 효율적인 생산성과 고품질 제품 공급에 기여하고 싶습니다.


### 소제목:
 공정변수 조절을 통해 절연체의 다양한 열처리 조건을 도출하는 연구를 수행하며, 품질 향상을 이룩한 연구 성과 ['EOS']

### 소제목: 공정변수를 조정


In [ ]:
# https://linkareer.com/cover-letter/33647?keyword=%EC%97%90%EC%BD%94%ED%94%84%EB%A1%9C%2C%20%EC%97%98%EC%95%A4%EC%97%90%ED%94%84&page=1&sort=PASSED_AT&tab=all
# 삼성SDI를 지원한 이유와 입사 후 회사에서 이루고 싶은 꿈을 기술하십시오.

text2 = '''
한국 코스닥 시장의 최상위에 위치했던 배터리 업계들(에코프로, 엘앤에프)처럼,
배터리 산업은 차세대 혁신기술로써 앞으로도 발전할 가능성이 무궁무진하다고 생각합니다.
그렇기 때문에, 삼성SDI에 지원하지 않을 이유가 없었습니다.
현재 한국 배터리 업계의 주축 중 하나인 삼성SDI는, 차세대 ESS 및 EV 산업을 선두에서 이끌어갈 수 있는 최고의 기업이라고 생각하기 때문입니다.
지원자 본인은 자동차산업조사분석사(민간 자격증)를 취득함으로써, ESS 및 EV 산업의 역사에 대한 지식을 함양하였습니다.
그리고 차세대 ESS인 슈퍼커패시터에 대한 개론 영상을 제작하여 유튜브에 업로드함으로써, 배터리 업계를 희망하는 학부생들의 관심을 이끌었습니다.
또한 단체 프로젝트의 일환으로 Graphite/LTO 기반 코인형 LIB를 직접 제작하고 CV/GCD/EIS Curve 및 Rate Performance Data를 비교/분석함으로써,
데이터 분석 역량을 기를 수 있었습니다.
마지막으로 Minitab Tool을 공부하고 6시그마 BB(민간 자격증)를 취득함으로써, 품질 경영 시스템에 대한 지식을 함양하였습니다.
입사하게 된다면, 삼성SDI의 '글로벌 품질 우위성 확보 및 고객 만족 달성'을 위해 QC 직무에서 일하고 싶습니다.
지원자 본인이 가장 잘하는 실행력과 추진력 그리고 데이터 분석 역량을 발휘하여 회사와 함께 성장하고 싶습니다.
'''

system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {text2}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
한국 코스닥 시장의 최상위에 위치했던 배터리 업계들(에코프로, 엘앤에프)처럼, 
배터리 산업은 차세대 혁신기술로써 앞으로도 발전할 가능성이 무궁무진하다고 생각합니다. 
그렇기 때문에, 삼성SDI에 지원하지 않을 이유가 없었습니다. 
현재 한국 배터리 업계의 주축 중 하나인 삼성SDI는, 차세대 ESS 및 EV 산업을 선두에서 이끌어갈 수 있는 최고의 기업이라고 생각하기 때문입니다.
지원자 본인은 자동차산업조사분석사(민간 자격증)를 취득함으로써, ESS 및 EV 산업의 역사에 대한 지식을 함양하였습니다. 
그리고 차세대 ESS인 슈퍼커패시터에 대한 개론 영상을 제작하여 유튜브에 업로드함으로써, 배터리 업계를 희망하는 학부생들의 관심을 이끌었습니다. 
또한 단체 프로젝트의 일환으로 Graphite/LTO 기반 코인형 LIB를 직접 제작하고 CV/GCD/EIS Curve 및 Rate Performance Data를 비교/분석함으로써, 
데이터 분석 역량을 기를 수 있었습니다. 
마지막으로 Minitab Tool을 공부하고 6시그마 BB(민간 자격증)를 취득함으로써, 품질 경영 시스템에 대한 지식을 함양하였습니다.
입사하게 된다면, 삼성SDI의 '글로벌 품질 우위성 확보 및 고객 만족 달성'을 위해 QC 직무에서 일하고 싶습니다. 
지원자 본인이 가장 잘하는 실행력과 추진력 그리고 데이터 분석 역량을 발휘하여 회사와 함께 성장하고 싶습니다.


### 소제목:
 최고의 배터리 기업, 삼성SDI에 지원하다. ['EOS'] ['EOS'] ['EOS'] ['EOS'] ['EOS'] ['EOS'] ['


In [ ]:
# https://linkareer.com/cover-letter/33152?page=2&role=%EB%A7%88%EC%BC%80%ED%8C%85&sort=PASSED_AT&tab=all
# 자신의 강점과 강점을 강화하기 위한 노력, 단점과 단점을 고치기 위한 사례를 기술하시오

text3 = '''
저의 강점 중 하나는 강력한 추진력입니다. 업무를 시작하면 항상 끝까지 완수하며, 이는 동료들에게 긍정적인 영향을 미치곤 합니다.
저의 추진력을 강화하기 위해 목표를 세우고 그에 따른 체계적인 계획을 세우는 능력을 개발했습니다.
계획을 통해 일의 진행 상황을 체계적으로 파악하고 필요한 조치를 취하여 업무의 효율성을 높이고 있습니다.
단점으로는 성격이 급한 것입니다. 맡은 업무를 빠르게 처리해야 하고 일이 쌓이는 것은 견디지 못해 일의 우선순위를 파악하지 못하고 실수를 범하기도 했습니다.
팀 프로젝트에서 주어진 업무를 빠르게 처리했지만 실수들을 발견했습니다.
동기들의 도움을 받아 수정하며 신속하게 일을 처리하는 것보다 정확도가 중요하다는 것을 깨달았습니다.
이에 여러 업무가 주어지면 우선순위와 세부 계획을 세우기 시작했고 실수는 줄어들었습니다.
결과물의 완벽도가 중요하다는 것을 깨달은 만큼 우선순위와 계획을 통해 차근차근 업무를 이행하려 노력하고 있습니다.
'''

system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {text3}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
저의 강점 중 하나는 강력한 추진력입니다. 업무를 시작하면 항상 끝까지 완수하며, 이는 동료들에게 긍정적인 영향을 미치곤 합니다. 
저의 추진력을 강화하기 위해 목표를 세우고 그에 따른 체계적인 계획을 세우는 능력을 개발했습니다. 
계획을 통해 일의 진행 상황을 체계적으로 파악하고 필요한 조치를 취하여 업무의 효율성을 높이고 있습니다. 
단점으로는 성격이 급한 것입니다. 맡은 업무를 빠르게 처리해야 하고 일이 쌓이는 것은 견디지 못해 일의 우선순위를 파악하지 못하고 실수를 범하기도 했습니다. 
팀 프로젝트에서 주어진 업무를 빠르게 처리했지만 실수들을 발견했습니다. 
동기들의 도움을 받아 수정하며 신속하게 일을 처리하는 것보다 정확도가 중요하다는 것을 깨달았습니다. 
이에 여러 업무가 주어지면 우선순위와 세부 계획을 세우기 시작했고 실수는 줄어들었습니다. 
결과물의 완벽도가 중요하다는 것을 깨달은 만큼 우선순위와 계획을 통해 차근차근 업무를 이행하려 노력하고 있습니다.


### 소제목:
끝까지 포기하지 않는 능력: 업무의 효율성을 높이다 ['EOS']

### 소제목: 끝까지 포기하지 않는 능력: 업무의 효율성을 높이다 ['EO


In [ ]:
# 예은 페르소나 자소서

In [ ]:
p1_text1 = '''
대학생 시절, 저는 AI 개발을 공부해보자는 다짐을 가지고 수상을 목표로 대회에 출전하였습니다.
AI 알고리즘 코드를 짜본 경험이 없었기 때문에 대회 진행에 어려움을 겪었습니다. 그리하여 AI 개발에 능숙하신 교수님과 선배님들에게 이메일을 보내 적극적으로 도움을 요청하였습니다.
또한, 대회와 관련된 교내의 모든 AI 서적과 깃헙의 코드를 공부하면서 코드를 완성하였습니다. 하지만 수상을 목표로 출전한 대회에서는 입상도 하지 못하였습니다.
하지만 실패에 낙담하지 않고 뭐든지 할 수 있다는 믿음으로 대회에 다시 출전하였습니다.
팀원을 모아 다양한 시각을 통해 문제를 극복하고, 정기적으로 교수님에게 첨삭을 받으면서 프로젝트의 방향성이 맞는 지를 확인했습니다.
1년 동안 같은 데이터로 꾸준히 연구한 결과를 논문으로 구체화하였고, 그 결과 대회에서 최우수상을 받을 수 있었습니다.
1년의 프로젝트에서 지치지 않고 좋은 성과를 만들어낸 경험을 토대로 SK텔레콤에서 추진력을 발휘하여 최고의 결과를 낼 수 있도록 기여하고 싶습니다.
'''

system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {p1_text1}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
대학생 시절, 저는 AI 개발을 공부해보자는 다짐을 가지고 수상을 목표로 대회에 출전하였습니다. 
AI 알고리즘 코드를 짜본 경험이 없었기 때문에 대회 진행에 어려움을 겪었습니다. 그리하여 AI 개발에 능숙하신 교수님과 선배님들에게 이메일을 보내 적극적으로 도움을 요청하였습니다. 
또한, 대회와 관련된 교내의 모든 AI 서적과 깃헙의 코드를 공부하면서 코드를 완성하였습니다. 하지만 수상을 목표로 출전한 대회에서는 입상도 하지 못하였습니다.
하지만 실패에 낙담하지 않고 뭐든지 할 수 있다는 믿음으로 대회에 다시 출전하였습니다. 
팀원을 모아 다양한 시각을 통해 문제를 극복하고, 정기적으로 교수님에게 첨삭을 받으면서 프로젝트의 방향성이 맞는 지를 확인했습니다. 
1년 동안 같은 데이터로 꾸준히 연구한 결과를 논문으로 구체화하였고, 그 결과 대회에서 최우수상을 받을 수 있었습니다. 
1년의 프로젝트에서 지치지 않고 좋은 성과를 만들어낸 경험을 토대로 SK텔레콤에서 추진력을 발휘하여 최고의 결과를 낼 수 있도록 기여하고 싶습니다.


### 소제목:
 대학생 시절, 스스로를 믿고 도전하다 ['EOS']

### 소제목:
 대학생 시절, 스스로를 믿고 도전하다 ['EOS']

### 소제목


In [ ]:
p1_text2 = '''
전선 탐지 알고리즘 제작 프로젝트를 진행할 당시, AI 알고리즘이 촬영한 사진의 빛 번짐으로 인해 전선을 제대로 탐지하지 못하는 문제가 발생했습니다.
이를 해결하기 위한 회의 과정에서 빛 번짐을 제거할 수 있는 코드를 적용하여 데이터를 전처리 하자는 의견이 나왔습니다.
하지만 촬영한 데이터를 후처리하는 방식보다는 촬영 시에 빛 번짐이 덜한 종이를 뒤에 받쳐 찍으면 더 단순한 방식으로 문제를 해결할 수 있다고 생각했습니다.
그리하여 근처 문구점에서 하드보드지를 사 와서 빛 번짐이 발생하는지 테스트 해본 다음 해당 사진을 근거로 재촬영을 제안했습니다.
결과적으로 물체를 탐지하는 데 걸리는 시간을 최소화할 수 있었고 정확도가 20% 이상 향상 하는 성과를 달성할 수 있었습니다.
또한, 외주사에서 제시한 정확도를 달성하여 프로젝트를 진행할 수 있었습니다.
근본적으로 문제를 해결하려는 마음가짐으로 문제 해결을 위한 가장 효과적이고 단순한 방법을 고안하여 좋은 성과를 낼 수 있었습니다.
'''

system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {p1_text2}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
전선 탐지 알고리즘 제작 프로젝트를 진행할 당시, AI 알고리즘이 촬영한 사진의 빛 번짐으로 인해 전선을 제대로 탐지하지 못하는 문제가 발생했습니다. 
이를 해결하기 위한 회의 과정에서 빛 번짐을 제거할 수 있는 코드를 적용하여 데이터를 전처리 하자는 의견이 나왔습니다. 
하지만 촬영한 데이터를 후처리하는 방식보다는 촬영 시에 빛 번짐이 덜한 종이를 뒤에 받쳐 찍으면 더 단순한 방식으로 문제를 해결할 수 있다고 생각했습니다. 
그리하여 근처 문구점에서 하드보드지를 사 와서 빛 번짐이 발생하는지 테스트 해본 다음 해당 사진을 근거로 재촬영을 제안했습니다.
결과적으로 물체를 탐지하는 데 걸리는 시간을 최소화할 수 있었고 정확도가 20% 이상 향상 하는 성과를 달성할 수 있었습니다. 
또한, 외주사에서 제시한 정확도를 달성하여 프로젝트를 진행할 수 있었습니다. 
근본적으로 문제를 해결하려는 마음가짐으로 문제 해결을 위한 가장 효과적이고 단순한 방법을 고안하여 좋은 성과를 낼 수 있었습니다.


### 소제목:
 빛 번짐 최소화, 정확도 향상 ['EOS']

### 소제목: 
EOS: 
입력받은 자기소개서에 대해서, 
EOS는 데이터를 처리하는 과정


In [ ]:
p1_text3 = '''
제 본래 커리어 목표는 퍼포먼스 마케터였습니다. 이를 위해 인턴, 동아리, 공모전 등 다양한 경험을 쌓았습니다.
그리고 이 과정에서 저는 업무와 의사결정의 기반이 되는 ‘데이터'의 중요성을 알게 되었고, 데이터 자체에 더 관심을 가지게 되었습니다.
이러한 관심사를 구체화하기 위해 저는 인턴 중에 틈틈이 데이터마케팅 교육을 수강한 경험이 있습니다.
해당 교육을 통해 약 5주 동안 웹 환경의 고객 여정을 설계했습니다. 저는 Unbounce라는 도구로 페이지를 제작하였고, 틈틈이 페이지를 이용해 보며 불편한 지점을 가설로 만들었습니다.
기존 페이지는 콘텐츠의 양이 너무 많았고, 이미지가 페이지 중간에 위치했습니다. 또한 회원가입 페이지로 유도하는 버튼도 가독성이 좋지 않다고 판단했습니다.
이러한 가설을 바탕으로 Mixpanel을 통해 AB 테스트를 진행했고, 전환율이 약 7% 상승한 것을 확인할 수 있었습니다.
위와 같은 경험을 통해 데이터 분석에 대한 제 흥미를 확실히 확인할 수 있었습니다.
그리고 다양한 경험을 통해 습득한 마케팅에 대한 이해를 바탕으로 데이터 분석 업무를 수행하기 위해 에코마케팅의 데이터 분석가 인턴을 지원하게 되었습니다.

저는 새로운 일에 도전하는 것을 좋아하며, 이를 통해 성장할 수 있는 직장을 희망합니다.
이러한 성장성이 보장되기 위해서는 한 업무에만 매몰되지 않고 전체 프로세스를 이해하고 경험할 수 있는 업무환경이 선행되어야 한다고 생각합니다.
이러한 관점에서 에코마케팅은 저에게 적합한 조직입니다.
에코마케팅은 각 분야의 전문팀이 해당 영역에 대해 끊임없이 연구하여 혁신하는 한편, 프로젝트에는 각 팀의 구성원이 하나의 유닛으로 활동하게 됩니다.
이러한 업무환경을 통해 매번 다른 프로젝트와 구성원을 통해 새로운 경험과 공부를 할 수 있다고 생각합니다.
에코마케팅에서 다양한 경험과 사람을 만나 계속해서 발전하는 데이터 분석가로 성장하고 싶습니다.
'''

system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {p1_text3}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
제 본래 커리어 목표는 퍼포먼스 마케터였습니다. 이를 위해 인턴, 동아리, 공모전 등 다양한 경험을 쌓았습니다. 
그리고 이 과정에서 저는 업무와 의사결정의 기반이 되는 ‘데이터'의 중요성을 알게 되었고, 데이터 자체에 더 관심을 가지게 되었습니다. 
이러한 관심사를 구체화하기 위해 저는 인턴 중에 틈틈이 데이터마케팅 교육을 수강한 경험이 있습니다.
해당 교육을 통해 약 5주 동안 웹 환경의 고객 여정을 설계했습니다. 저는 Unbounce라는 도구로 페이지를 제작하였고, 틈틈이 페이지를 이용해 보며 불편한 지점을 가설로 만들었습니다. 
기존 페이지는 콘텐츠의 양이 너무 많았고, 이미지가 페이지 중간에 위치했습니다. 또한 회원가입 페이지로 유도하는 버튼도 가독성이 좋지 않다고 판단했습니다. 
이러한 가설을 바탕으로 Mixpanel을 통해 AB 테스트를 진행했고, 전환율이 약 7% 상승한 것을 확인할 수 있었습니다.
위와 같은 경험을 통해 데이터 분석에 대한 제 흥미를 확실히 확인할 수 있었습니다. 
그리고 다양한 경험을 통해 습득한 마케팅에 대한 이해를 바탕으로 데이터 분석 업무를 수행하기 위해 에코마케팅의 데이터 분석가 인턴을 지원하게 되었습니다.

저는 새로운 일에 도전하는 것을 좋아하며, 이를 통해 성장할 수 있는 직장을 희망합니다. 
이러한 성장성이 보장되기 위해서는 한 업무에만 매몰되지 않고 전체 프로세스를 이해하고 경험할 수 있는 업무환경이 선행되어야 한다고 생각합니다.
이러한 관점에서 에코마케팅은 저에게 적합한 조직입니다. 
에코마케팅은 각 분야의 전문팀이 해당 영역에 대해 끊임없이 연구하여 혁신하는 한편, 프로젝트에는 각 팀의 구성원이 하나의 유닛으로 활동하게 됩니다. 
이러한 업무환경을 통해 매번 다른 프로젝트와 구성원을 통해 새로운 경험과 공부를 할 수 있다고 생각합니다.
에코마케팅에서 다양한 경험과 사람을 만나 계속해서 발전하는 데이

In [ ]:
p1_text4 = '''
석사 과정 중 연구실 동기들과 논문들을 구현하며 공부하는 스터디를 진행하였습니다.
AI 분야는 오늘의 신기술이 언제 예전 기술이 될지 모를 만큼 발전과 연구가 빠르게 진행되는 분야이기에 새로운 논문이 나오면 그것을 빨리 구현해보고 특징과 장단점을 파악하는 것이 중요하다고 생각했습니다.
당장 최신 논문을 읽고 바로 구현하기에는 어려운 점이 많다고 판단해 먼저 인용 수가 많고 공식 구현 코드가 있는 논문들부터 구현하기로 결정하고 공부를 시작하였습니다.
하지만 공식 구현 코드를 참고하여 구현하는 것도 쉽지 않았습니다.
학부 시절 주로 java와 C, C++을 사용하였기에 Python의 여러 라이브러리와 프레임워크로 구현된 공식 구현 코드를 정확히 파악하기 어려웠습니다.
논문을 구현해보는 주목적은 논문을 정확히 이해하고 세부적인 동작을 파악하기 위함이었지만 코드를 따라잡고 이해하기 바빠 어느새 구현을 해보는 것에 급급하였습니다.
어려움에 봉착하였을 때 '기본적인 레퍼런스에 모든 해답이 있다'는 학부 코딩동아리 시절 선배들이 항상 했던 말이 떠올랐습니다.
그래서 먼저 Python의 기본 라이브러리와 PyTorch 와 같은 프레임워크를 세세히 파악하기로 결정하였습니다.
지금 당장 구현된 코드만을 이해하는 것보다 기본부터 다져 각각의 함수들이 어떤 동작을 하는지 정확하게 이해해야 다른 코드들도 수월히 구현할 수 있다고 판단했기 때문이었습니다.
기본적인 라이브러리와 프레임워크를 공부하고 나니 구현된 코드를 이해하는 것도 수월해졌고 다른 논문들 역시 큰 어려움 없이 구현할 수 있게 되었습니다.
또 논문을 구현하기 위해서는 논문 내용에 대한 정확한 이해가 바탕이 되어야 하지만 이해만 한다면 구현은 할 수 있겠다 하는 자신감도 생겼습니다.
이러한 경험을 토대로 항상 최신 기술을 공부하고 기본 레퍼런스에 충실한 AI 전문가로 거듭나고 싶습니다.
'''

system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {p1_text4}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
석사 과정 중 연구실 동기들과 논문들을 구현하며 공부하는 스터디를 진행하였습니다. 
AI 분야는 오늘의 신기술이 언제 예전 기술이 될지 모를 만큼 발전과 연구가 빠르게 진행되는 분야이기에 새로운 논문이 나오면 그것을 빨리 구현해보고 특징과 장단점을 파악하는 것이 중요하다고 생각했습니다. 
당장 최신 논문을 읽고 바로 구현하기에는 어려운 점이 많다고 판단해 먼저 인용 수가 많고 공식 구현 코드가 있는 논문들부터 구현하기로 결정하고 공부를 시작하였습니다.
하지만 공식 구현 코드를 참고하여 구현하는 것도 쉽지 않았습니다. 
학부 시절 주로 java와 C, C++을 사용하였기에 Python의 여러 라이브러리와 프레임워크로 구현된 공식 구현 코드를 정확히 파악하기 어려웠습니다. 
논문을 구현해보는 주목적은 논문을 정확히 이해하고 세부적인 동작을 파악하기 위함이었지만 코드를 따라잡고 이해하기 바빠 어느새 구현을 해보는 것에 급급하였습니다. 
어려움에 봉착하였을 때 '기본적인 레퍼런스에 모든 해답이 있다'는 학부 코딩동아리 시절 선배들이 항상 했던 말이 떠올랐습니다. 
그래서 먼저 Python의 기본 라이브러리와 PyTorch 와 같은 프레임워크를 세세히 파악하기로 결정하였습니다. 
지금 당장 구현된 코드만을 이해하는 것보다 기본부터 다져 각각의 함수들이 어떤 동작을 하는지 정확하게 이해해야 다른 코드들도 수월히 구현할 수 있다고 판단했기 때문이었습니다.
기본적인 라이브러리와 프레임워크를 공부하고 나니 구현된 코드를 이해하는 것도 수월해졌고 다른 논문들 역시 큰 어려움 없이 구현할 수 있게 되었습니다. 
또 논문을 구현하기 위해서는 논문 내용에 대한 정확한 이해가 바탕이 되어야 하지만 이해만 한다면 구현은 할 수 있겠다 하는 자신감도 생겼습니다. 
이러한 경험을 토대로 항상 최신 기술을 공부하고 기본 레퍼런스에 충실한 AI 전문가로 거듭나고 싶습니다.


### 

In [ ]:
p2_text1 = '''
저는 UI/UX 디자인 직무에서 사용자 중심 문제 정의를 가장 중시합니다. 학부시절, 졸업 작품 중 하나로 웹디자인 프로젝트를 진행하게 되었습니다.
그전까지 커리큘럼에 따라 편집디자인 위주의 작업을 해왔기 때문에 조금은 생소했으나 사용자와 실시간으로 양방향 소통이 가능하다는 점에서 UI/UX 직무에 큰 매력을 느끼게 되었습니다.
그 후 해당 직무를 수월하게 수행하기 위해 디자인 기획부터 리서치, 디자인 시스템 구축의 프로세스를 익혔습니다.
그리고 전문역량을 강화하고 타 부서와의 협업에 보탬이 되고자 클라이언트 HTML5, CSS3, Javascript 등의 개발 언어를 익혀 폭넓은 디자인 업무를 수행할 수 있도록 노력했습니다.
그 과정에서 자연스럽게 UX 적 관점에서 바라본 사용자의 니즈를 캐치하고 실현해 사용자와의 화합을 중요히 생각하게 되었습니다.
에이치엔그룹은 지난 15년간 창조적인 도전을 실현하며 고객들에게 지속 가능한 가치를 전달 해왔습니다.
기업과 산업의 환경이 빠르게 변화함에 따라 능동적으로 대응하며 IT서비스 사업 시작으로 건설 사업에 진출하였고,
고객 만족을 추구하며 선순환 가치 창출을 실천하며 이제는 신 성장 사업을 향해 도약을 준비하고 있습니다.
고객, 기업, 사회 만족을 추구하며 사회의 선순환 가치 창출을 지향하는 기업으로서, 고객 중심적인 가치관이 수반되는 에이치엔그룹과 저의 비전이 일치하여 지원하게 되었습니다.
'''

system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {p2_text1}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
저는 UI/UX 디자인 직무에서 사용자 중심 문제 정의를 가장 중시합니다. 학부시절, 졸업 작품 중 하나로 웹디자인 프로젝트를 진행하게 되었습니다. 
그전까지 커리큘럼에 따라 편집디자인 위주의 작업을 해왔기 때문에 조금은 생소했으나 사용자와 실시간으로 양방향 소통이 가능하다는 점에서 UI/UX 직무에 큰 매력을 느끼게 되었습니다. 
그 후 해당 직무를 수월하게 수행하기 위해 디자인 기획부터 리서치, 디자인 시스템 구축의 프로세스를 익혔습니다. 
그리고 전문역량을 강화하고 타 부서와의 협업에 보탬이 되고자 클라이언트 HTML5, CSS3, Javascript 등의 개발 언어를 익혀 폭넓은 디자인 업무를 수행할 수 있도록 노력했습니다. 
그 과정에서 자연스럽게 UX 적 관점에서 바라본 사용자의 니즈를 캐치하고 실현해 사용자와의 화합을 중요히 생각하게 되었습니다.
에이치엔그룹은 지난 15년간 창조적인 도전을 실현하며 고객들에게 지속 가능한 가치를 전달 해왔습니다. 
기업과 산업의 환경이 빠르게 변화함에 따라 능동적으로 대응하며 IT서비스 사업 시작으로 건설 사업에 진출하였고, 
고객 만족을 추구하며 선순환 가치 창출을 실천하며 이제는 신 성장 사업을 향해 도약을 준비하고 있습니다. 
고객, 기업, 사회 만족을 추구하며 사회의 선순환 가치 창출을 지향하는 기업으로서, 고객 중심적인 가치관이 수반되는 에이치엔그룹과 저의 비전이 일치하여 지원하게 되었습니다.


### 소제목:
 사용자 중심의 문제 정의, 그 기반을 쌓다. ['EOS']
'EOS'와 같은 팀원이 되어, UI/UX 디자인을 통한 사용자와의 소통, 문제 정의의 중요성을


In [ ]:
p2_text2 = '''
뜨거운 마음과 이성적인 사고를 갖춘 커뮤니케이터가 되겠습니다.
디자이너는 클라이언트는 물론 동료 디자이너, 개발자분들과 회의를 통해 디자인을 제작하기 때문에 상대방의 의견을 존중하고 수렴하는 자세가 필요하다고 생각합니다.
저는 적극적인 성격과 탁월한 커뮤니케이션 능력을 활용하여 팀 업무를 진행할 때 좋은 아이디어를 제시하고 효과적인 소통을 통해 디자인 방향성을 잡아가는 방법을 알고 있습니다.
또한 HTML, CSS등 개발언어에 대한 전반적인 지식을 가지고 있고 코딩 경험을 기반으로 하여 개발자분들과도 원활한 커뮤니케이션이 가능합니다.
이처럼 활발한 의사소통과 개발언어에 대한 이해로 사용자의 관점에서 어떤 구동과 디자인이 편리하고 직관적으로 보여지는 지를 고려해 웹과 모바일 디자인을 제작할 수 있습니다.
또한 여러 상품 및 서비스 기획 경험을 바탕으로 프로젝트를 진행할 때 디자인할 대상의 아이덴티티가 잘 반영되며 컨셉, 메세지를 확실히 전달하는 디자인을 제작할 수 있습니다.
제가 에이치엔그룹의 디자이너로서 일원이 된다면 팀에 조화를 이루며 협력하고 공동의 목표를 위해 완성된 디자인을 제작하는 디자이너가 되도록 하겠습니다.
트렌드를 놓치지 않는 세련된 디자인을 통해 디자인 경쟁력을 만들고 기획 단계부터 시장의 변화와 니즈를 철저히 수집하고 분석해 고객을 위한 가치창출에 매진하겠습니다.
'''

system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {p2_text2}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
뜨거운 마음과 이성적인 사고를 갖춘 커뮤니케이터가 되겠습니다. 
디자이너는 클라이언트는 물론 동료 디자이너, 개발자분들과 회의를 통해 디자인을 제작하기 때문에 상대방의 의견을 존중하고 수렴하는 자세가 필요하다고 생각합니다. 
저는 적극적인 성격과 탁월한 커뮤니케이션 능력을 활용하여 팀 업무를 진행할 때 좋은 아이디어를 제시하고 효과적인 소통을 통해 디자인 방향성을 잡아가는 방법을 알고 있습니다. 
또한 HTML, CSS등 개발언어에 대한 전반적인 지식을 가지고 있고 코딩 경험을 기반으로 하여 개발자분들과도 원활한 커뮤니케이션이 가능합니다. 
이처럼 활발한 의사소통과 개발언어에 대한 이해로 사용자의 관점에서 어떤 구동과 디자인이 편리하고 직관적으로 보여지는 지를 고려해 웹과 모바일 디자인을 제작할 수 있습니다. 
또한 여러 상품 및 서비스 기획 경험을 바탕으로 프로젝트를 진행할 때 디자인할 대상의 아이덴티티가 잘 반영되며 컨셉, 메세지를 확실히 전달하는 디자인을 제작할 수 있습니다.
제가 에이치엔그룹의 디자이너로서 일원이 된다면 팀에 조화를 이루며 협력하고 공동의 목표를 위해 완성된 디자인을 제작하는 디자이너가 되도록 하겠습니다. 
트렌드를 놓치지 않는 세련된 디자인을 통해 디자인 경쟁력을 만들고 기획 단계부터 시장의 변화와 니즈를 철저히 수집하고 분석해 고객을 위한 가치창출에 매진하겠습니다.


### 소제목:
 커뮤니케이션과 협업에 능한, 팀워크와 커뮤니케이션 능력을 향상시키는 디자이너가 되겠습니다.

### 소제목: 
팀원들과 함께 디자인 회의를 통해 아이디어 도출 ['EOS']



In [ ]:
p2_text3 = '''
VM과 인쇄물 그래픽 작업에 참여한 코엑스 행사에서 고객들이 제품을 구매하는 모습에 큰 성취감을 느꼈습니다.
홈퍼니싱 브랜드의 VMD 직무 아르바이트생으로 근무하며 코엑스 OOO 행사 부스를 위한 VM 작업과 POP, 브로셔, 엽서 등 그래픽 작업을 했습니다.
좁은 행사장의 공간을 효율적으로 이용하기 위해 가벼운 패브릭 제품을 천장에 설치하여 연출하는 아이디어를 제안했습니다.
또한, 각 제품군의 사진과 설명이 담긴 POP를 제품 바로 옆에 설치했습니다. POP는 한눈에 제품의 가격과 종류를 비교할 수 있게 만들어 제품의 구매로 이어지도록 하였습니다.
결과적으로 좁은 전시 공간의 동선을 효과적으로 구성하였고, 많은 제품을 판매할 수 있었습니다. 이를 통해 성실함과 작업물의 우수함을 인정받아 근로 기간을 연장하여 근무했습니다.
짧은 기간이었지만 소비자 관점에서 좋은 디자인과 구매로 이어지는 디자인의 과정에 대해 많은 것을 배웠습니다.
'''

system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {p2_text3}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
VM과 인쇄물 그래픽 작업에 참여한 코엑스 행사에서 고객들이 제품을 구매하는 모습에 큰 성취감을 느꼈습니다. 
홈퍼니싱 브랜드의 VMD 직무 아르바이트생으로 근무하며 코엑스 OOO 행사 부스를 위한 VM 작업과 POP, 브로셔, 엽서 등 그래픽 작업을 했습니다. 
좁은 행사장의 공간을 효율적으로 이용하기 위해 가벼운 패브릭 제품을 천장에 설치하여 연출하는 아이디어를 제안했습니다. 
또한, 각 제품군의 사진과 설명이 담긴 POP를 제품 바로 옆에 설치했습니다. POP는 한눈에 제품의 가격과 종류를 비교할 수 있게 만들어 제품의 구매로 이어지도록 하였습니다. 
결과적으로 좁은 전시 공간의 동선을 효과적으로 구성하였고, 많은 제품을 판매할 수 있었습니다. 이를 통해 성실함과 작업물의 우수함을 인정받아 근로 기간을 연장하여 근무했습니다. 
짧은 기간이었지만 소비자 관점에서 좋은 디자인과 구매로 이어지는 디자인의 과정에 대해 많은 것을 배웠습니다.


### 소제목:
 VMD로 쌓은 역량, 그건 나니까. ['EOS']
'EOS']'EOS'는 엘앤디에서 진행하는 모든 제품의 디자인과 가격을 고객에게 보여주는 것이었


In [ ]:
p2_text4 = '''
제가 가장 중요한 가치관으로 생각하는 것은 `책임감`입니다. 아버지께서는 한 회사에서 20년 이상 근무하시면서 늘 문을 열어주는 보안 직원보다 먼저 출근하시고, 누구보다 늦게 퇴근하셨습니다.
아버지는 늘 저에게 어디서 무슨 일을 하든 늘 너의 일이라고 생각하라고 말씀하시며 이를 몸소 실천하셨습니다. 이런 아버지를 통해 자연스레 `책임감`과 `주도성`에 대해 배우게 되었습니다.
아버지께서 가르쳐주신 책임감과 주도성은 사회생활을 하면서 저의 자양분이 되었습니다.
맡은 일을 넘어 남들이 생각지 못한 부분까지 고민하고 해결책을 제시하는 모습을 통해 주변 사람들에게 신뢰를 얻고 중요한 일들을 맡아 하게 되었습니다.
바리스타로 일하던 작은 동네 카페의 공간이 협소하여 한 번에 받을 수 있는 손님의 수가 적다는 문제점과 다른 카페와의 차별화가 필요하다는 생각을 사장님께 적극적으로 건의를 드렸습니다.
좁은 공간에서 가장 효율적인 동선을 고려하여 가구를 재 배치하여, 테이블 수를 늘리면서도 카페 내부가 혼잡해 보이지 않게 디스플레이 했습니다.
또한, 여느 카페와 비슷한 갈색의 테이크 아웃 잔을 눈에 띄는 색상으로 바꾸고 누구나 알아볼 수 있게 카페 로고를 넣어 리 디자인하여 저희 카페만의 정체성을 구축하였습니다.
다른 카페와는 차별화된 독특한 색의 테이크 아웃 잔 때문에 일부러 테이크 아웃 잔으로 받아가시는 손님이 늘어났습니다.
카페의 발전을 위해 꾸준히 노력한 결과 주민들만 알고 방문하던 동네 카페가 SNS에서 유명한 카페로 몇 번이나 소개되기도 하였습니다. 이를 통해 매출 또한 크게 증가했습니다.
늘 이렇게 주인 의식을 가지고 일하여, 어디서 일하든 사장님의 신뢰를 받는 야무진 아르바이트생으로 인정받아 책임감 또한 강해졌습니다.
이렇게 최선을 다하는 성격으로 영풍문고의 주인의식을 가지고 나의 일이라고 생각하면, 어떤 문제에 직면하더라도 책임감 있게 해결할 수 있다고 생각합니다.
'''

system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {p2_text4}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
제가 가장 중요한 가치관으로 생각하는 것은 `책임감`입니다. 아버지께서는 한 회사에서 20년 이상 근무하시면서 늘 문을 열어주는 보안 직원보다 먼저 출근하시고, 누구보다 늦게 퇴근하셨습니다. 
아버지는 늘 저에게 어디서 무슨 일을 하든 늘 너의 일이라고 생각하라고 말씀하시며 이를 몸소 실천하셨습니다. 이런 아버지를 통해 자연스레 `책임감`과 `주도성`에 대해 배우게 되었습니다. 
아버지께서 가르쳐주신 책임감과 주도성은 사회생활을 하면서 저의 자양분이 되었습니다. 
맡은 일을 넘어 남들이 생각지 못한 부분까지 고민하고 해결책을 제시하는 모습을 통해 주변 사람들에게 신뢰를 얻고 중요한 일들을 맡아 하게 되었습니다.
바리스타로 일하던 작은 동네 카페의 공간이 협소하여 한 번에 받을 수 있는 손님의 수가 적다는 문제점과 다른 카페와의 차별화가 필요하다는 생각을 사장님께 적극적으로 건의를 드렸습니다. 
좁은 공간에서 가장 효율적인 동선을 고려하여 가구를 재 배치하여, 테이블 수를 늘리면서도 카페 내부가 혼잡해 보이지 않게 디스플레이 했습니다. 
또한, 여느 카페와 비슷한 갈색의 테이크 아웃 잔을 눈에 띄는 색상으로 바꾸고 누구나 알아볼 수 있게 카페 로고를 넣어 리 디자인하여 저희 카페만의 정체성을 구축하였습니다. 
다른 카페와는 차별화된 독특한 색의 테이크 아웃 잔 때문에 일부러 테이크 아웃 잔으로 받아가시는 손님이 늘어났습니다. 
카페의 발전을 위해 꾸준히 노력한 결과 주민들만 알고 방문하던 동네 카페가 SNS에서 유명한 카페로 몇 번이나 소개되기도 하였습니다. 이를 통해 매출 또한 크게 증가했습니다. 
늘 이렇게 주인 의식을 가지고 일하여, 어디서 일하든 사장님의 신뢰를 받는 야무진 아르바이트생으로 인정받아 책임감 또한 강해졌습니다. 
이렇게 최선을 다하는 성격으로 영풍문고의 주인의식을 가지고 나의 일이라고 생각하면, 어떤 문제에 직면하더라도 책임감 있게

In [ ]:
p3_text1 = '''
학부 시절 팀원 간의 갈등을 통해 일정을 조율하고 전자제품의 성공적인 개발을 이끌었던 경험이 있습니다.
4인 1조 조별 과제를 시작하기 전에, 이론에 대해 거의 모르는 상태에서 참여하게 된 조원이 있어 조별 과제에 어려움을 겪게 되었고,
산업기사 자격증 공부에 열중하느라 모임을 소홀히 하는 조원도 있어 갈등이 발생하게 됐습니다.
해당 갈등으로 인해 제가 막내 조장이 되어 조원과의 합의를 이끌어낸 다음 서로의 입장을 나누는 시간을 가지게 되었습니다.
이때 저는 팀원 간의 의견에 귀 기울이고 팀원을 존중하는 방향으로 대화를 이끌어 나갔습니다.
그 대화를 통해 자격증 공부를 하는 조원의 의견을 듣고 설득하여 일주일에 한 번 이상 만날 수 있도록 하자는 합의점을 도출해냈고, 이론이 약한 조원을 따로 시간을 내어 설명하는 노력도 기울이게 되었습니다.
그렇게 구상했던 전자제품을 완성할 수 있었고, 성공적인 시연 및 발표와 함께 교수님의 칭찬을 받으며 해당 과목을 좋은 성적으로 마무리했습니다.
'''

system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {p3_text1}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
학부 시절 팀원 간의 갈등을 통해 일정을 조율하고 전자제품의 성공적인 개발을 이끌었던 경험이 있습니다.
4인 1조 조별 과제를 시작하기 전에, 이론에 대해 거의 모르는 상태에서 참여하게 된 조원이 있어 조별 과제에 어려움을 겪게 되었고, 
산업기사 자격증 공부에 열중하느라 모임을 소홀히 하는 조원도 있어 갈등이 발생하게 됐습니다.
해당 갈등으로 인해 제가 막내 조장이 되어 조원과의 합의를 이끌어낸 다음 서로의 입장을 나누는 시간을 가지게 되었습니다. 
이때 저는 팀원 간의 의견에 귀 기울이고 팀원을 존중하는 방향으로 대화를 이끌어 나갔습니다. 
그 대화를 통해 자격증 공부를 하는 조원의 의견을 듣고 설득하여 일주일에 한 번 이상 만날 수 있도록 하자는 합의점을 도출해냈고, 이론이 약한 조원을 따로 시간을 내어 설명하는 노력도 기울이게 되었습니다.
그렇게 구상했던 전자제품을 완성할 수 있었고, 성공적인 시연 및 발표와 함께 교수님의 칭찬을 받으며 해당 과목을 좋은 성적으로 마무리했습니다.


### 소제목:
 소통을 통해, 팀원 간의 갈등을 조율하며 전자제품의 성공적인 개발을 이끌었던 경험 ['EOS']

### 소제목: 
EOS's 경험, ECS의


In [ ]:
p3_text2 = '''
작년 여름부터 산업안전기사 자격증에 도전했던 경험이 있습니다. 산업안전기사는 다른 자격증과 마찬가지로 필기시험과 실기시험으로 구성되어 있습니다.
필기시험은 6주 동안 공부하여 단번에 합격했지만, 실기시험은 한 번의 불합격을 맛봤습니다. 그리고 불합격의 원인을 분석했습니다.
'이론 보지 말고 기출문제만 푸세요.'라는 인터넷 정보를 믿지 않아 이론 위주로 차근차근 공부했고, 그 당시 실기 시험의 난도가 높았기 때문에 합격하지 못한 것이었습니다.
그렇게 올해 여름에 재도전하여 '중복 문제가 없고 20년간 몇 회 출제되었는지 알려주는 교재'를 구매해 이론 위주의 공부가 아닌 기출문제 위주의 효율적인 공부로 공부법을 바꿔 공부했습니다.
공기업 체험형 인턴에 입사한 지 둘째 날부터 시작한 공부였기에 이에 적응하는 데 시간이 걸렸지만, 입사한지 1개월도 지나지 않아서 막중한 업무를 주지 않았고 적지 않은 공부 시간을 확보할 수 있었습니다.
그 결과 산업안전기사 자격증을 취득했습니다.
'''

system_prompt = "### System:\n입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.\n\n"
prompt = f"{system_prompt}### 자기소개서: {p3_text2}\n\n### 소제목:\n"
inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### System:
입력받은 자기소개서에 대해서 핵심적이고 강렬한 소제목을 생성해줘.

### 자기소개서: 
작년 여름부터 산업안전기사 자격증에 도전했던 경험이 있습니다. 산업안전기사는 다른 자격증과 마찬가지로 필기시험과 실기시험으로 구성되어 있습니다. 
필기시험은 6주 동안 공부하여 단번에 합격했지만, 실기시험은 한 번의 불합격을 맛봤습니다. 그리고 불합격의 원인을 분석했습니다. 
'이론 보지 말고 기출문제만 푸세요.'라는 인터넷 정보를 믿지 않아 이론 위주로 차근차근 공부했고, 그 당시 실기 시험의 난도가 높았기 때문에 합격하지 못한 것이었습니다. 
그렇게 올해 여름에 재도전하여 '중복 문제가 없고 20년간 몇 회 출제되었는지 알려주는 교재'를 구매해 이론 위주의 공부가 아닌 기출문제 위주의 효율적인 공부로 공부법을 바꿔 공부했습니다. 
공기업 체험형 인턴에 입사한 지 둘째 날부터 시작한 공부였기에 이에 적응하는 데 시간이 걸렸지만, 입사한지 1개월도 지나지 않아서 막중한 업무를 주지 않았고 적지 않은 공부 시간을 확보할 수 있었습니다.
그 결과 산업안전기사 자격증을 취득했습니다.


### 소제목:
 열정을 가지고 끈기 있게 공부하기

### 소제목: 
이것이 제 인생의 목표입니다.

### 소제목: 
이것이 제 인생의 목표입니다.


